In [30]:

import sys
import os
import random
import glob
import torch
sys.path.append("C:/Users/yuuta/Documents/fashion")
from utils.util import filter_basic_items, open_json
from utils.util import is_target_category
from utils.util import calculate_euclid_sum
from utils.infer import id_to_vector
from utils.util import calc_roc_auc


In [31]:
import importlib
import utils
importlib.reload(utils.util)

<module 'utils.util' from 'C:\\Users/yuuta/Documents/fashion\\utils\\util.py'>

In [32]:
if torch.cuda.is_available():
    device = torch.device("cuda")  # GPUデバイスを取得
else:
    device = torch.device("cpu")  # CPUデバイスを取得

In [33]:
negative_path = glob.glob(
    "C:/Users/yuuta/Documents/fashion/data/negative_coordinates_random/**.json"
    # negative_coordinates_random
)
positive_path = glob.glob(
    "C:/Users/yuuta/Documents/fashion/data/train/**/**.json"
)

In [34]:
print(len(negative_path), len(positive_path))

109997 162975


In [35]:
negative_path = random.sample(negative_path, 100000)
positive_path = random.sample(positive_path, 100000)
labels = []
proposal_score = []
positive_ave = 0
negative_ave = 0

In [36]:
p_cnt = 0
for fp in positive_path:
    json_dict = open_json(fp)
    items = filter_basic_items(json_dict["items"])
    attributes = []
    vectors = []
    for item in items:
        try:
            itemId = str(item["itemId"])
        except Exception as e:
            print(fp, " : ", e)
            continue

        vector = id_to_vector(itemId)
        vectors.append(torch.tensor(vector))
    if len(vectors) != 3:
        continue
    ps = calculate_euclid_sum(vectors)
    proposal_score.append(ps.to("cpu"))
    p_cnt += 1
    labels.append(0)
    positive_ave += ps

positive_ave /= p_cnt
print(f"positive fin average: {positive_ave}")

C:\Users\yuuta\AppData\Local\Temp\ipykernel_17220\683951227.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  vectors.append(torch.tensor(vector))


positive fin average: tensor([[301.3525]], device='cuda:0')


In [37]:
n_cnt = 0
for fp in negative_path:
    json_dict = open_json(fp)
    if json_dict == None:
        continue
    parent_dir = os.path.dirname(fp)
    items = []
    attributes = []
    vectors = []
    for item in filter(is_target_category, json_dict["items"]):
        try:
            itemId = str(item["itemId"])
        except Exception as e:
            print(fp, " : ", e)
            continue

        # vectorを推定
        vector = id_to_vector(itemId)
        vectors.append(torch.tensor(vector))
    if len(vectors) != 3:
        continue
    ps = calculate_euclid_sum(vectors)
    proposal_score.append(ps.to("cpu"))
    labels.append(1)
    n_cnt += 1
    negative_ave += ps

negative_ave /= n_cnt

print(positive_ave, negative_ave)

C:\Users\yuuta\AppData\Local\Temp\ipykernel_17220\3424496347.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  vectors.append(torch.tensor(vector))


tensor([[301.3525]], device='cuda:0') tensor([[300.9420]], device='cuda:0')


In [38]:
print(len(proposal_score), len(labels))

148268 148268


In [39]:
p_auc = calc_roc_auc(labels, proposal_score, "proposal_model_image_only")
print(p_auc)

ValueError: Found array with dim 3. None expected <= 2.

In [ ]:
print(p_auc)

0.5058069849142385
